# SiFThon - سيفثون
أداة تنصيب سورس سيفثون على حساب التلجرام

## المتطلبات الأساسية
- API ID
- API Hash
- كود الجلسة
- توكن البوت

In [ ]:
# إعداد المتغيرات
session = input('أدخل كود الجلسة: ').strip()
api_id = input('أدخل API ID: ').strip()
api_hash = input('أدخل API Hash: ').strip()
bot_token = input('أدخل توكن البوت: ').strip()

# حفظ المتغيرات في env
with open('env', 'w') as f:
    f.write(f'API_ID={api_id}\nAPI_HASH={api_hash}\nSESSION={session}\nBOT_TOKEN={bot_token}\n')
print('✅ تم حفظ المتغيرات في ملف env')

In [ ]:
# رفع ملف env موجود
from google.colab import files
upload = input('هل تريد رفع ملف env موجود؟ (y/n): ').strip().lower()
if upload == 'y':
    uploaded = files.upload()
    for filename in uploaded.keys():
        if filename.startswith('env') or filename == '.env':
            import shutil
            shutil.move(filename, 'env')
            print('✅ تم رفع ملف env بنجاح')

In [ ]:
# تثبيت المتطلبات
!pip install telethon pyrogram tgcrypto python-dotenv requests yt-dlp pillow pymongo sqlalchemy psutil --quiet

In [ ]:
# تحميل وفك سورس SiFThon
import requests, zipfile, os
url = 'https://hasting.free.nf/SiFThon.zip'
r = requests.get(url)
with open('SiFThon.zip','wb') as f: f.write(r.content)
with zipfile.ZipFile('SiFThon.zip','r') as z: z.extractall('SiFThon')
os.remove('SiFThon.zip')
print('✅ تم تحميل وفك ضغط السورس')

In [ ]:
# نسخ ملف env داخل المجلد
import shutil
shutil.copy('env','SiFThon/.env')
print('✅ تم نسخ env إلى مجلد SiFThon')

In [ ]:
# تشغيل السورس
os.chdir('SiFThon')
if os.path.exists('main.py'):
    !python main.py
elif os.path.exists('__main__.py'):
    !python -m SiFThon
else:
    print('❌ ملف التشغيل الرئيسي غير موجود')

## انتهى التنصيب
يمكنك الآن استخدام SiFThon على حسابك.